**Tabajo Practico 1 - Pandas & Spark RDD**

Cada ejercicio se considera independiente uno del otro, no hay variables que se sobreescriban entre ejercicios, se crean nuevas.

# Instalamos e importamos librerías

In [85]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Datos
import numpy as np
import pandas as pd

# Autenticamos con Google Drive

In [86]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos los archivos csv

In [87]:
downloaded = drive.CreateFile({'id':"1fYyEYw0JCjk6k94bO_GANtS83DofQGp1"})   # replace the id with id of file you want to access
downloaded.GetContentFile('googleplaystore.csv')

In [88]:
downloaded = drive.CreateFile({'id':"1ViZVfHBZe84ZMWZ0nGUx8eL2gUNZ2hwg"})   # replace the id with id of file you want to access
downloaded.GetContentFile('googleplaystore_user_reviews.csv')

# Analisis exploratorio de datos

In [89]:
df_app = pd.read_csv("googleplaystore.csv").drop_duplicates()
df_app.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


- googleplaystore.csv

| Columna | Descripción |Tipo de dato | Comentarios |
|---------|-------------|-------------|------------|
| App | Nombre de la aplicación | object (cadena de texto) | |
| Category | Categoría a la que pertenece la aplicación | category | |
| Rating | Calificación general de los usuarios de la aplicación | float32 (número decimal) | |
| Reviews | Número de reseñas de usuarios para la aplicación | int64 (número entero) | |
| Size | Tamaño de la aplicación | int (número entero) unidad (Mega) | 0 = Varies with device|
| Installs | Número de descargas/instalaciones de usuarios para la aplicación | int64 (número entero) | |
| Type | Tipo de aplicación (gratuita o de pago) | category | |
| Price | Precio de la aplicación | float32 (número flotante) | |
| Content Rating | Grupo de edad al que está dirigida la aplicación: niños / mayores de 21 años / adultos | category | |
| Genres | Una aplicación puede pertenecer a varios géneros (aparte de su categoría principal) | object (cadena de texto)| |
| Last Updated | Fecha actualizada en la que la aplicación se actualizó por última vez en Play Store | datetime64[ns] (fecha y hora) | |
| Current Ver | Versión actual de la aplicación disponible en Play Store | object (cadena de texto) | |
| Android Ver | Versión mínima de Android requerida para ejecutar la aplicación | object (cadena de texto) ||

In [90]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10358 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10358 non-null  object 
 1   Category        10358 non-null  object 
 2   Rating          8893 non-null   float64
 3   Reviews         10358 non-null  object 
 4   Size            10358 non-null  object 
 5   Installs        10358 non-null  object 
 6   Type            10357 non-null  object 
 7   Price           10358 non-null  object 
 8   Content Rating  10357 non-null  object 
 9   Genres          10358 non-null  object 
 10  Last Updated    10358 non-null  object 
 11  Current Ver     10350 non-null  object 
 12  Android Ver     10355 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


- Calcular el número de NaN por columna

In [91]:
nan_count = df_app.isnull().sum()
print(nan_count[nan_count > 0])

Rating            1465
Type                 1
Content Rating       1
Current Ver          8
Android Ver          3
dtype: int64


In [92]:
df_reviews = pd.read_csv("googleplaystore_user_reviews.csv").drop_duplicates()
df_reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


# A) Preprocesamiento los tipos de datos

Se sabe que la columna "Reviews" debe ser de tipo númerico entero positivo.
Al momento de convertirla a numerica, con astype, se procuce un error.
Se encuentra un fila que tiene todos sus valores incorrectos.

In [93]:
df_app[df_app["Reviews"].str.isdecimal() == False]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


- Borramos la ttoda fila que contine el valor "3.0M"

In [94]:
df_app = df_app[df_app["Reviews"] != '3.0M']

## Columna `Category`

In [95]:
print("Cantidad de caterias en 'Category':", df_app["Category"].value_counts().count())
df_app["Category"] = df_app["Category"].astype("category")

Cantidad de caterias en 'Category': 33


## Columna `Rating`

In [96]:
df_app['Rating'] = df_app['Rating'].astype("float32")

## Columna `Rewiers`

In [97]:
df_app["Reviews"] = df_app["Reviews"].astype('int32')

## Columna `Size`

- Columna "Size" es de tipo numérica.Se reemplaza los "Varies with device" por 0.

In [98]:
df_app["Size"] = df_app["Size"].replace("Varies with device", 0)

-  Función que elimina la letra "M" y "k" y lo convierte a float32 con la unidad de medida en Mega.

In [99]:
def convert_to_mega(value):
    if isinstance(value, str):
        if 'M' in value:
            return float(value.replace('M', ''))
        elif 'k' in value:
            return float(value.replace('k', '')) / 1000
    return float(value) / 1000000

In [100]:
df_app['Size'] = df_app['Size'].map(convert_to_mega)
df_app["Size"] = df_app["Size"].astype("float32")

## Columna `Installs`

- Funcion que elimina los sibolos [',', '+'] y retorna un int.

In [101]:
def convert_to_int(value):
    if isinstance(value, str):
        return int(value.replace(',', '').replace('+', ''))
    return int(value)

In [102]:
df_app['Installs'] = df_app['Installs'].map(convert_to_int)

## Columna `Type`

In [103]:
df_app["Type"] = df_app["Type"].astype("category")

## Columna `Price`

- La columna "Price" es de tipo object, pero tiene valores numéricos y letras (\$). Elimino el signo "$" y la convertimos a numérica.

In [104]:
df_app["Price"] = df_app["Price"].astype("str").str.replace("$", "")
df_app["Price"] = df_app["Price"].astype("str").astype("float32")

<ipython-input-104-e4466724b8a8>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_app["Price"] = df_app["Price"].astype("str").str.replace("$", "")


## Columna `Content Rating`

In [105]:
df_app["Content Rating"] = df_app["Content Rating"].astype("category")

## Columna `Last Updated` (fecha)

In [106]:
df_app['Last Updated'] = pd.to_datetime(df_app['Last Updated'])

# Pandas (⭐)

## 9) ¿Cuál es la aplicación que generó más dinero? (⭐)

- Función que calcula el dinero generado por cada aplicación.

In [107]:
def revenue(row):
  return row["Price"] * row["Installs"]

 - Se obtiene la primera fila con el mayor valor de revenue. Al ser una funcion nativa de python es más eficiente.

In [108]:
df_app_top = max(df_app.iterrows(), key=lambda x: revenue(x[1]))[1]

- Mostrar la información de la aplicaación que generó más dinero

In [109]:
print(df_app_top)

App                               Minecraft
Category                             FAMILY
Rating                                  4.5
Reviews                             2376564
Size                                    0.0
Installs                           10000000
Type                                   Paid
Price                                  6.99
Content Rating                 Everyone 10+
Genres            Arcade;Action & Adventure
Last Updated            2018-07-24 00:00:00
Current Ver                         1.5.2.1
Android Ver              Varies with device
Name: 2241, dtype: object


## 11) ¿Cuál es la aplicación con mayor promedio de score de sentimiento subjetivo? (⭐)

In [110]:
df_app_grouped = df_reviews.groupby("App")

df_app_mean = df_app_grouped["Sentiment_Subjectivity"].mean()

- Se usa la función max de Python sobre los valores del dataFrame con el argumento key igual a la función mean. Al ser una funcion nativa de python es más eficiente.

In [111]:
df_app_top = max(df_app_mean.items(), key=lambda x: x[1])

In [112]:
print("name: ", df_app_top[0], "\n", "mean:", df_app_top[1])

name:  Google Slides 
 mean: 0.9166666666666666


# Pandas (⭐⭐)

## 24) Indica las 10 apps de categoría Sport con sentimiento positivo y mayor rating.(⭐⭐)

In [113]:
df_sport = df_app[(df_app["Category"] == "SPORTS") & (df_app["Rating"].notnull())][["App","Rating"]].drop_duplicates()

In [114]:
df_positive_reviews = df_reviews[df_reviews['Sentiment'] == 'Positive'][["App"]].drop_duplicates()

- Hacemos un merge de los dos dataframes, sobre la columna "App" y "inner" = interseccion.


In [115]:
df_merged = pd.merge(df_sport, df_positive_reviews, on="App", how="inner")

In [116]:
df_sport_positive_sorted = df_merged.sort_values(by="Rating", ascending=False)
df_sport_positive_sorted.head(10)

,App,Rating
32,Dream League Soccer 2018,4.6
20,FanDuel: Daily Fantasy Sports,4.6
17,Golf GPS Rangefinder: Golf Pad,4.6
15,Golf GPS by SwingxSwing,4.6
25,850 Sports News Digest,4.6
6,All Football - Latest News & Videos,4.6
26,365Scores - Live Scores,4.6
8,"All Football GO- Live Score, Games",4.6
14,"GolfNow: Tee Time Deals at Golf Courses, Golf GPS",4.5
27,Cricbuzz - Live Cricket Scores & News,4.5


## 28) Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo. (⭐⭐)

- La variable "Size" los datos que tienen 0 = "Varies with device". Y "groupby" ignora los valores NaN y los valores de "Size" no tienen NaN.

In [117]:
df_app[df_app["Size"] != 0].groupby("Android Ver")[["Size"]].mean()

,Size
Android Ver,
1.0 and up,3.855500
1.5 and up,5.071631
1.6 and up,3.119138
2.0 and up,6.256806
2.0.1 and up,22.222141
2.1 and up,5.584451
2.2 - 7.1.1,5.100000
2.2 and up,8.220924
2.3 and up,20.831053


## 36) Devolver las categorías que tengan una app dominante de nivel K. Una app es dominante a nivel K si la cantidad de descargas es mayor al número de de descarga de las k siguientes apps ordenadas según el número de descargas. (⭐⭐)

- Función para determinar si una aplicación es dominante a nivel K en su categoría

In [118]:
def es_dominante(df):
    return df['Installs'] > df['Installs'].shift(-k)

In [119]:
# Elegir el valor de K
k = 500

- Ordenar el DataFrame por categoría y número de descargas

In [120]:
df_app_sort = df_app.sort_values(['Category', 'Installs'], ascending=[True, False], inplace=False)

- Agrupar el DataFrame por categoría y aplicar la función es_dominante

In [121]:
apps_dominantes = df_app_sort.groupby('Category').apply(es_dominante)

- Filtrar las aplicaciones dominantes

In [122]:
apps_dominantes = apps_dominantes[apps_dominantes]

- Obtener las categorías que tienen aplicaciones dominantes

In [123]:
categorias_con_apps_dominantes = apps_dominantes.index.get_level_values(0).unique()

print("Categorias con App dominante K=", k,":", list(categorias_con_apps_dominantes))

Categorias con App dominante K= 500 : ['FAMILY', 'GAME', 'TOOLS']
